In [ ]:
%pip install openai langchain-openai dotenv

# Multi-agent 
- Agent 1: Section Extractor GROBID
- Agent 2: Section Summariser
- Agent 3: Summary Aggregator

## Section Extractor GROBID

In [5]:
import requests
from lxml import etree

class GrobidSectionAgent:
    def __init__(self, grobid_url: str = "http://localhost:8070/api/processFulltextDocument"):
        self.grobid_url = grobid_url

    def extract_sections(self, pdf_path: str):
        with open(pdf_path, "rb") as f:
            response = requests.post(self.grobid_url, files={"input": f})
        response.raise_for_status()
        xml_text = response.text
        sections = self._parse_tei_xml(xml_text)
        flat_sections = self._flatten_sections(sections)
        return [s for s in flat_sections if s["content"].strip()]

    def _parse_tei_xml(self, xml_text: str):
        root = etree.fromstring(xml_text.encode("utf-8"))
        body = root.find(".//{*}body")
        sections = []
        if body is not None:
            for div in body.findall("{*}div"):
                sections.append(self._parse_div(div))
        return sections

    def _parse_div(self, div):
        heading = div.findtext("{*}head")
        content = []
        current_page = None

        for el in div.iter():
            # Capture page number from <pb n="..."/>
            if el.tag.endswith("pb") and "n" in el.attrib:
                current_page = int(el.attrib["n"])
            # Capture text
            if el.text:
                content.append(el.text)

        content_str = " ".join(content).strip()
        subsections = [self._parse_div(d) for d in div.findall("{*}div")]

        return {
            "heading": heading.strip() if heading else None,
            "content": content_str,
            "page_start": current_page,
            "subsections": subsections
        }

    def _flatten_sections(self, sections):
        flat = []

        def walk(sec, parent=None):
            title = sec.get("heading") or parent or "Untitled"
            flat.append({
                "heading": title,
                "content": sec.get("content", ""),
                "page_start": sec.get("page_start")
            })
            for s in sec.get("subsections", []):
                walk(s, title)

        for s in sections:
            walk(s)

        return flat


## Section Summary Agent

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

class SectionSummaryAgent:
    def __init__(self, llm_model="gpt-4o-mini", temperature=0.05):
        self.llm = ChatOpenAI(model=llm_model, temperature=temperature)
        # Define prompt template
        self.prompt_template = PromptTemplate.from_template("""
You are an expert AI research assistant. Summarize the section "{section_name}" 
from the following content. Produce a concise, structured, and academic-style summary.

Section Text:
{section_text}

Instructions:
- Include key points and methodology if present
- Mention any datasets, results, or experiments
- Keep summary factual and concise
""")
        self.parser = StrOutputParser()

    def summarize(self, section_name, section_text):
        # Fill prompt
        prompt_input = {
            "section_name": section_name,
            "section_text": section_text
        }
        # Generate summary using LLM
        summary = self.prompt_template | self.llm | self.parser
        return summary.invoke(prompt_input)

c:\Users\Yue Ning\Desktop\MiscLearning\ai_researcher_database\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Summary Aggregator Agent

In [12]:

class SummaryAggregatorAgent:
    def __init__(self, llm_model="gpt-4o-mini", temperature=0.05):
        self.llm = ChatOpenAI(model=llm_model, temperature=temperature)
        # Define prompt template
        self.prompt_template = PromptTemplate.from_template("""
You are an expert AI research assistant. Your task is to combine the following individual section summaries into one cohesive and well-structured academic summary of the research paper in JSON format.

Guidelines:
- Reorganize and merge overlapping or overly detailed sections as needed.
- Choose the most logical and meaningful section headings yourself (e.g., Introduction, Methods, Results, Discussion, Conclusion), based on content.
- Ensure smooth transitions and consistent academic tone throughout.
- Preserve key technical details, results, and findings from the input summaries.
- Do not add information that is not supported by the summaries.

Input Section Summaries:
{sections_text}

Output:
A coherent and complete academic-style summary written in paragraphs with appropriate section headings.
- Use this schema (keys may be omitted if not applicable):
{
  "sections": [ { "heading": "string", "content": "string" } ],
  "global_conclusion": "string"
}
""")
        self.parser = StrOutputParser()

    def combine(self, section_summaries: list[dict]) -> dict:
        sections_text = "\n\n".join(
            [f"## {s['section']}\n{s['summary']}" for s in section_summaries]
        )
        chain = self.prompt_template | self.llm | self.parser
        raw = chain.invoke({"sections_text": sections_text})
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            # Fallback: wrap as plain text if model ever violates JSON-only
            return {"sections": [{"heading": "Combined", "content": raw}], "global_conclusion": ""}

# Final Integration

In [13]:
### to add in .py file 

# from section_detection_agent import GrobidSectionAgent

pdf_path = "papers/hospital_bed_capacity_planning.pdf"

# Initialize GROBID agent
grobid_agent = GrobidSectionAgent()
sections = grobid_agent.extract_sections(pdf_path)
print(sections)

# Initialize your summarization agent
section_summarizer = SectionSummaryAgent()

summaries = []
for section in sections:
    name = section.get("heading") or "Unnamed Section"
    text = section.get("content", "")
    
    if not text.strip():
        continue  # skip empty sections
    
    print(f"Summarizing: {name}...")
    summary = section_summarizer.summarize(name, text)
    summaries.append({"section": name, "summary": summary})

# Initialize aggregator agent 
summary_aggregator = SummaryAggregatorAgent()
final_summary = summary_aggregator.combine(summaries)



[{'heading': 'Introduction', 'content': 'Introduction The Hospital Bed Capacity (HBC) forecasting problem has taken significant attention because of its effects on the sustainability of hospitals, particularly in terms of hospital economic efficiency, and patient satisfaction  [1] [2] [3] [4] [5] [6] [7] [8] [9] [10] [11] [12] [13] [14] [15] [16] [17] [18]', 'page_start': None}, {'heading': 'Literature review', 'content': "Literature review The literature on healthcare centers' bed capacity forecasting is reviewed to clear the paper's contributions and novelty aspects.  Bachouch et al. (2012) [22] [11] Devapriya et al. (2015) [23] Keegan et al. (2018) The emergency department plays a vital role in the hospital because it is the first deal for patients. Therefore, the effective management of the emergency department is significant in improving the quality of service and treatment. Nas and Koyuncu (2019) used ten ML tools to predict the patients' arrival rates. They applied a simulation 

KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "sections"\'}.  Expected: [\'\\n  "sections"\', \'sections_text\'] Received: [\'sections_text\']\nNote: if you intended {\n  "sections"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "sections"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [9]:
summaries

[{'section': 'Introduction',
  'summary': '**Summary of the Introduction Section**\n\nThe Hospital Bed Capacity (HBC) forecasting problem is increasingly recognized for its critical implications on hospital sustainability, economic efficiency, and patient satisfaction. The section highlights the importance of accurate HBC forecasting in optimizing resource allocation and improving healthcare delivery. Although specific methodologies, datasets, results, or experiments are not detailed in this section, the emphasis on the multifaceted impact of HBC forecasting underscores its relevance in contemporary healthcare management. The references cited indicate a broad spectrum of research supporting the significance of this issue.'},
 {'section': 'Literature review',
  'summary': '**Literature Review Summary**\n\nThe literature on forecasting bed capacity in healthcare centers highlights the significance of effective management, particularly in emergency departments, to enhance service quality.

In [10]:
final_summary # most recent output 

"# Summary of Research on Hospital Bed Capacity Forecasting\n\n## Introduction\nThe forecasting of Hospital Bed Capacity (HBC) is increasingly recognized as a critical factor influencing hospital sustainability, economic efficiency, and patient satisfaction. Accurate HBC forecasting is essential for optimizing resource allocation and enhancing healthcare delivery. This research underscores the multifaceted impact of HBC forecasting, highlighting its relevance in contemporary healthcare management.\n\n## Literature Review\nThe existing literature emphasizes the importance of effective bed capacity management, particularly in emergency departments, to improve service quality. Key studies, such as those by Bachouch et al. (2012), Devapriya et al. (2015), and Keegan et al. (2018), illustrate the emergency department's role as the initial point of patient care. Additionally, Nas and Koyuncu (2019) employed machine learning tools to predict patient arrival rates and optimize bed requirements

Output to be displayed: `output['final_summary]`

In [ ]:
import json
output = {
    "sections": summaries,            # list of {"section": str, "summary": str}
    "final_summary": final_summary,   # str
}

# Print JSON to stdout
print(json.dumps(output, ensure_ascii=False, indent=2))

# Optionally save JSON to a file
with open("multi_agent_output.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

{
  "sections": [
    {
      "section": "Introduction",
      "summary": "**Summary of the Introduction Section**\n\nThe Hospital Bed Capacity (HBC) forecasting problem is increasingly recognized for its critical implications on hospital sustainability, economic efficiency, and patient satisfaction. The section highlights the importance of accurate HBC forecasting in optimizing resource allocation and improving healthcare delivery. Although specific methodologies, datasets, results, or experiments are not detailed in this section, the emphasis on the multifaceted impact of HBC forecasting underscores its relevance in contemporary healthcare management. The references cited indicate a broad spectrum of research supporting the significance of this issue."
    },
    {
      "section": "Literature review",
      "summary": "**Literature Review Summary**\n\nThe literature on forecasting bed capacity in healthcare centers highlights the significance of effective management, particularly in